In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import sys
sys.path.append("../")
# from util import get_data_from_api
# data = get_data_from_api("INSERT_USER/INSERT_RUN_ID")
# df = pd.DataFrame(data)
# df.to_csv("50k.csv", index=False)

In [3]:
from util import model_translations

df = pd.read_csv("50k.csv")
df.model = df.model.map(model_translations)
df.loss = df.model_id.map(lambda x: x.split("(")[0])
df = df.drop(columns=["model_id", "samples"])
df["loss"] = df["loss"] + " " + df["lambda"].map(str)
df["loss"] = df["loss"].map(lambda x: x.replace("nan", "").strip())
df = df.drop(columns=["lambda"])

dataset_translation = {
    "sarcastic-headlines": "sarcastic",
    "sst2": "sst2",
}
df.dataset = df.dataset.map(dataset_translation)
df["model_for_data"] = df["model"] + "_" + df["dataset"]
df = df.drop(columns=["model", "dataset", "combined"])  # not interested in the combined score here
df

,loss,polarity,semantic,model_for_data
0,MultipleNegatives,0.740,0.426,minilm-6_sst2
1,OnlineContrastive 1.0,0.827,0.325,minilm-6_sst2
2,OnlineContrastive 0.75,0.818,0.322,minilm-6_sst2
3,OnlineContrastive 0.5,0.803,0.317,minilm-6_sst2
4,OnlineContrastive 0.25,0.785,0.314,minilm-6_sst2
...,...,...,...,...
131,Triplet 7.5,0.887,0.745,gte-base_sarcastic
132,Triplet 5.0,0.887,0.745,gte-base_sarcastic
133,Triplet 1.0,0.900,0.766,gte-base_sarcastic
134,Triplet 0.1,0.907,0.774,gte-base_sarcastic


In [ ]:
_df = df.groupby(["model_for_data", "loss"]).mean()
_df = _df.reset_index()
_df = _df.sort_values(by="loss", ascending=True)
_df

,model_for_data,loss,polarity,semantic
0,e5-small_sarcastic,Contrastive 0.1,0.910,0.791
51,gte-base_sst2,Contrastive 0.1,0.909,0.805
85,gte-small_sst2,Contrastive 0.1,0.893,0.818
34,gte-base_sarcastic,Contrastive 0.1,0.901,0.746
102,minilm-6_sarcastic,Contrastive 0.1,0.836,0.215
...,...,...,...,...
50,gte-base_sarcastic,Triplet 7.5,0.887,0.745
33,e5-small_sst2,Triplet 7.5,0.908,0.819
16,e5-small_sarcastic,Triplet 7.5,0.892,0.784
118,minilm-6_sarcastic,Triplet 7.5,0.853,0.201


In [5]:
# ensure polarity and semantic are floats
_df = _df.astype({"polarity": str, "semantic": str})

In [6]:
def transpose_and_group(df, metric="polarity", as_latex=False):
    tmp = df.pivot(index="model_for_data", columns="loss", values=metric).reset_index().T
    column_names = tmp.iloc[0]
    tmp = tmp.drop(tmp.index[0])
    idx = tmp.index
    tmp = tmp.reset_index(drop=True)
    tmp.columns = column_names

    tmp["loss"] = idx
    tmp["lambda"] = tmp["loss"].map(lambda x: float(x.split(" ")[1]) if len(x.split(" ")) > 1 else "-")
    tmp["loss"] = tmp["loss"].map(lambda x: x.split(" ")[0])
    tmp = tmp[["loss", "lambda"] + list(tmp.columns[:-2])]
    tmp = tmp.sort_values(by=["loss", "lambda"], ascending=True)
    tmp.columns.name = None

    if as_latex:
        return tmp.style.to_latex()
    return tmp

# pd.DataFrame(transpose_and_group(_df, metric="polarity"))
print(transpose_and_group(_df, metric="polarity", as_latex=True))

\begin{tabular}{lllllllllll}
 & loss & lambda & e5-small_sarcastic & e5-small_sst2 & gte-base_sarcastic & gte-base_sst2 & gte-small_sarcastic & gte-small_sst2 & minilm-6_sarcastic & minilm-6_sst2 \\
0 & Contrastive & 0.100000 & 0.91 & 0.919 & 0.901 & 0.909 & 0.878 & 0.893 & 0.836 & 0.738 \\
1 & Contrastive & 0.250000 & 0.914 & 0.921 & 0.905 & 0.909 & 0.892 & 0.9 & 0.843 & 0.766 \\
2 & Contrastive & 0.500000 & 0.916 & 0.924 & 0.905 & 0.91 & 0.897 & 0.9 & 0.849 & 0.8 \\
3 & Contrastive & 0.750000 & 0.915 & 0.925 & 0.904 & 0.913 & 0.896 & 0.902 & 0.855 & 0.816 \\
4 & Contrastive & 1.000000 & 0.915 & 0.924 & 0.906 & 0.913 & 0.897 & 0.904 & 0.863 & 0.825 \\
5 & MultipleNegatives & - & 0.786 & 0.833 & 0.79 & 0.859 & 0.772 & 0.837 & 0.75 & 0.74 \\
6 & OnlineContrastive & 0.100000 & 0.915 & 0.925 & 0.904 & 0.908 & 0.886 & 0.897 & 0.842 & 0.76 \\
7 & OnlineContrastive & 0.250000 & 0.917 & 0.927 & 0.905 & 0.912 & 0.896 & 0.901 & 0.847 & 0.785 \\
8 & OnlineContrastive & 0.500000 & 0.917 & 0.926 &

In [7]:
print(transpose_and_group(_df, metric="semantic", as_latex=True))

\begin{tabular}{lllllllllll}
 & loss & lambda & e5-small_sarcastic & e5-small_sst2 & gte-base_sarcastic & gte-base_sst2 & gte-small_sarcastic & gte-small_sst2 & minilm-6_sarcastic & minilm-6_sst2 \\
0 & Contrastive & 0.100000 & 0.791 & 0.832 & 0.746 & 0.805 & 0.772 & 0.818 & 0.215 & 0.316 \\
1 & Contrastive & 0.250000 & 0.794 & 0.836 & 0.752 & 0.81 & 0.779 & 0.823 & 0.217 & 0.314 \\
2 & Contrastive & 0.500000 & 0.795 & 0.835 & 0.757 & 0.813 & 0.785 & 0.825 & 0.21 & 0.309 \\
3 & Contrastive & 0.750000 & 0.797 & 0.835 & 0.758 & 0.813 & 0.784 & 0.825 & 0.218 & 0.31 \\
4 & Contrastive & 1.000000 & 0.795 & 0.835 & 0.761 & 0.813 & 0.782 & 0.824 & 0.231 & 0.312 \\
5 & MultipleNegatives & - & 0.829 & 0.851 & 0.801 & 0.825 & 0.815 & 0.838 & 0.385 & 0.426 \\
6 & OnlineContrastive & 0.100000 & 0.799 & 0.836 & 0.752 & 0.81 & 0.78 & 0.823 & 0.223 & 0.308 \\
7 & OnlineContrastive & 0.250000 & 0.803 & 0.838 & 0.761 & 0.814 & 0.787 & 0.825 & 0.225 & 0.314 \\
8 & OnlineContrastive & 0.500000 & 0.805 & 